<a href="https://colab.research.google.com/github/JordiCorbilla/ocular-disease-intelligent-recognition-deep-learning/blob/master/odir_inception_v3_training_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# odir inception_v3 training basic

In [20]:
# Imports
import os
import tensorflow as tf
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.python.keras.optimizer_v1 import SGD

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
import secrets
import odir
from odir_advance_plotting import Plotter
from odir_kappa_score import FinalScore
from odir_predictions_writer import Prediction
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
from sklearn.utils import class_weight
import numpy as np

In [21]:
# main parameters
batch_size = 32
num_classes = 8
epochs = 100
patience = 5

In [22]:
# Create test run folder where to store the details of the run
token = secrets.token_hex(16)
folder = r'test_run'
new_folder = os.path.join(folder, token)

if not os.path.exists(new_folder):
    os.makedirs(new_folder) 

In [23]:
# Base Model definition for Inception V3
base_model = inception_v3.InceptionV3
base_model = base_model(weights='imagenet', include_top=False)   

In [24]:
# Model summary
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_94 (Conv2D)             (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_94 (BatchN  (None, None, None,   96         ['conv2d_94[0][0]']              
 ormalization)                  32)                                                         

                                                                                                  
 average_pooling2d_9 (AveragePo  (None, None, None,   0          ['max_pooling2d_5[0][0]']        
 oling2D)                       192)                                                              
                                                                                                  
 conv2d_99 (Conv2D)             (None, None, None,   12288       ['max_pooling2d_5[0][0]']        
                                64)                                                               
                                                                                                  
 conv2d_101 (Conv2D)            (None, None, None,   76800       ['activation_100[0][0]']         
                                64)                                                               
                                                                                                  
 conv2d_10

                                                                                                  
 conv2d_112 (Conv2D)            (None, None, None,   16384       ['average_pooling2d_10[0][0]']   
                                64)                                                               
                                                                                                  
 batch_normalization_106 (Batch  (None, None, None,   192        ['conv2d_106[0][0]']             
 Normalization)                 64)                                                               
                                                                                                  
 batch_normalization_108 (Batch  (None, None, None,   192        ['conv2d_108[0][0]']             
 Normalization)                 64)                                                               
                                                                                                  
 batch_nor

                                                                                                  
 batch_normalization_119 (Batch  (None, None, None,   192        ['conv2d_119[0][0]']             
 Normalization)                 64)                                                               
                                                                                                  
 activation_113 (Activation)    (None, None, None,   0           ['batch_normalization_113[0][0]']
                                64)                                                               
                                                                                                  
 activation_115 (Activation)    (None, None, None,   0           ['batch_normalization_115[0][0]']
                                64)                                                               
                                                                                                  
 activatio

                                128)                                                              
                                                                                                  
 conv2d_130 (Conv2D)            (None, None, None,   114688      ['activation_129[0][0]']         
                                128)                                                              
                                                                                                  
 batch_normalization_125 (Batch  (None, None, None,   384        ['conv2d_125[0][0]']             
 Normalization)                 128)                                                              
                                                                                                  
 batch_normalization_130 (Batch  (None, None, None,   384        ['conv2d_130[0][0]']             
 Normalization)                 128)                                                              
          

...

 batch_normalization_147 (Batch  (None, None, None,   576        ['conv2d_147[0][0]']             
 Normalization)                 192)                                                              
                                                                                                  
 batch_normalization_152 (Batch  (None, None, None,   576        ['conv2d_152[0][0]']             
 Normalization)                 192)                                                              
                                                                                                  
 batch_normalization_153 (Batch  (None, None, None,   576        ['conv2d_153[0][0]']             
 Normalization)                 192)                                                              
                                                                                                  
 activation_144 (Activation)    (None, None, None,   0           ['batch_normalization_144[0][0]']
          

 conv2d_154 (Conv2D)            (None, None, None,   147456      ['mixed6[0][0]']                 
                                192)                                                              
                                                                                                  
 conv2d_157 (Conv2D)            (None, None, None,   258048      ['activation_156[0][0]']         
                                192)                                                              
                                                                                                  
 conv2d_162 (Conv2D)            (None, None, None,   258048      ['activation_161[0][0]']         
                                192)                                                              
                                                                                                  
 conv2d_163 (Conv2D)            (None, None, None,   147456      ['average_pooling2d_15[0][0]']   
          

 batch_normalization_165 (Batch  (None, None, None,   960        ['conv2d_165[0][0]']             
 Normalization)                 320)                                                              
                                                                                                  
 batch_normalization_169 (Batch  (None, None, None,   576        ['conv2d_169[0][0]']             
 Normalization)                 192)                                                              
                                                                                                  
 activation_165 (Activation)    (None, None, None,   0           ['batch_normalization_165[0][0]']
                                320)                                                              
                                                                                                  
 activation_169 (Activation)    (None, None, None,   0           ['batch_normalization_169[0][0]']
          

                                384)                                                              
                                                                                                  
 activation_173 (Activation)    (None, None, None,   0           ['batch_normalization_173[0][0]']
                                384)                                                              
                                                                                                  
 activation_176 (Activation)    (None, None, None,   0           ['batch_normalization_176[0][0]']
                                384)                                                              
                                                                                                  
 activation_177 (Activation)    (None, None, None,   0           ['batch_normalization_177[0][0]']
                                384)                                                              
          

 Normalization)                 384)                                                              
                                                                                                  
 batch_normalization_186 (Batch  (None, None, None,   1152       ['conv2d_186[0][0]']             
 Normalization)                 384)                                                              
                                                                                                  
 conv2d_187 (Conv2D)            (None, None, None,   393216      ['average_pooling2d_17[0][0]']   
                                192)                                                              
                                                                                                  
 batch_normalization_179 (Batch  (None, None, None,   960        ['conv2d_179[0][0]']             
 Normalization)                 320)                                                              
          

In [25]:
# Plot the model
tf.keras.utils.plot_model(model, to_file=os.path.join(new_folder, 'model_inception_v3.png'), show_shapes=True, show_layer_names=True)

In [26]:
# metrics
defined_metrics = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
]

In [27]:
# SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
print('Configuration Start -------------------------')
print(sgd.get_config())
print('Configuration End -------------------------')
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=defined_metrics)

Configuration Start -------------------------
{'name': 'SGD', 'learning_rate': 0.01, 'decay': 1e-06, 'momentum': 0.9, 'nesterov': True}
Configuration End -------------------------


In [28]:
# Train the data
(x_train, y_train), (x_test, y_test) = odir.load_data(224)

In [29]:
# Define class names
x_test_drawing = x_test

x_train = inception_v3.preprocess_input(x_train)
x_test = inception_v3.preprocess_input(x_test)

class_names = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'AMD', 'Hypertension', 'Myopia', 'Others']

In [30]:
# Plot data input
plotter = Plotter(class_names)

In [31]:
# Early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min', verbose=1)

In [33]:
# Train the model
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True, #class_weight= class_weight,
                    validation_data=(x_test, y_test), callbacks=[callback])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.